In [16]:
# 1. 导入所需要的库
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [17]:
# 2. 下载CIFAR-10数据集
# 设计图像预处理：图像增强 + 转换为张量 + 标准化
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


In [18]:

# 下载训练集和测试集
trainset = torchvision.datasets.CIFAR10(root='CIFAR10', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='CIFAR10', train=False,
                                        download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                          shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [1:55:27<00:00, 24612.53it/s]  


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10
Files already downloaded and verified


In [21]:
# 3.使用自己的CNN模型作为训练网络
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64 * 8 * 8, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
model = SimpleCNN(num_classes=10)

In [22]:
# 4. 从头训练的自己的CNN网络
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 迁移到GPU上（）如果有的话
device = torch.device("cude:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 训练网络
for epoch in range(10): # 就演示训练10个epochs
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取输入数据
        inputs, labels = data[0].to(device), data[1].to(device)

        # 清零参数梯度
        optimizer.zero_grad()

        # 前向 + 反向 + 优化
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199: # 每200批次打印一次
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Training Finished')

[1,   200] loss: 2.283
[1,   400] loss: 2.180
[1,   600] loss: 2.069
[2,   200] loss: 1.927
[2,   400] loss: 1.887
[2,   600] loss: 1.837
[3,   200] loss: 1.741
[3,   400] loss: 1.706
[3,   600] loss: 1.670
[4,   200] loss: 1.626
[4,   400] loss: 1.605
[4,   600] loss: 1.580
[5,   200] loss: 1.555
[5,   400] loss: 1.548
[5,   600] loss: 1.525
[6,   200] loss: 1.499
[6,   400] loss: 1.500
[6,   600] loss: 1.493
[7,   200] loss: 1.462
[7,   400] loss: 1.458
[7,   600] loss: 1.444
[8,   200] loss: 1.430
[8,   400] loss: 1.412
[8,   600] loss: 1.444
[9,   200] loss: 1.389
[9,   400] loss: 1.382
[9,   600] loss: 1.393
[10,   200] loss: 1.363
[10,   400] loss: 1.339
[10,   600] loss: 1.364
Training Finished


In [24]:
# 5. 测试网络性能
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test iamges: %d %%' % (
    100 * correct / total
))


Accuracy of the network on the 10000 test iamges: 52 %
